In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import openai
import os

# Load the dataset (with correct image_url column)
df = pd.read_csv('your csv file.csv')

# Convert 'rating' column to numeric, invalid parsing will be set as NaN
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
# Extract numbers from 'number_of_reviews' and convert them to numeric
df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'].str.extract(r'(\d+)')[0], errors='coerce')
# Ensure comments and image URLs are valid (in case there are missing values)
df['comments'] = df['comments'].fillna("No comments available")
df['image_url'] = df['image_url'].fillna("https://via.placeholder.com/150")

# Remove duplicates based on restaurant name
df = df.drop_duplicates(subset='rest_name')

# Initialize the Dash app
rest_dash = dash.Dash(__name__)

# Layout of the app with a San Francisco-themed background
rest_dash.layout = html.Div(style={
    'width': '80%', 
    'margin': 'auto', 
    'font-family': 'Arial, sans-serif',
    'background-image': 'url("https://upload.wikimedia.org/wikipedia/commons/a/a0/San_Francisco_skyline.jpg")',  # San Francisco-themed background
    'background-size': 'cover',
    'padding': '20px'
}, children=[
    html.H1("Restaurant Dashboard", style={'text-align': 'center', 'color': '#007BFF'}),
    
    html.Div([
        # Filters section on the left
        html.Div(style={
            'width': '40%', 
            'display': 'inline-block', 
            'vertical-align': 'top', 
            'padding': '20px', 
            'border': '1px solid #ccc', 
            'border-radius': '10px', 
            'background-color': 'rgba(255, 255, 255, 0.8)'
        }, children=[
            html.Label("Select Rating"),
            dcc.Slider(
                id='rating-slider',
                min=0,
                max=5,
                step=0.1,
                value=4,  # Default rating filter value
                marks={i: f'{i}' for i in range(6)},
                tooltip={"placement": "bottom", "always_visible": True}
            ),
            dcc.Dropdown(
                id='food-type-dropdown',
                options=[{'label': food, 'value': food} for food in df['food_type'].unique()],
                value=None,
                placeholder="Select food type"
            ),
            dcc.Dropdown(
                id='price-range-dropdown',
                options=[{'label': price, 'value': price} for price in df['coupon'].unique()],
                value=None,
                placeholder="Select price range"
            ),
            dcc.Dropdown(
                id='number-reviews-dropdown',
                options=[{'label': f'At least {i}', 'value': i} for i in sorted(df['number_of_reviews'].dropna().unique())],
                value=None,
                placeholder="Select minimum number of reviews"
            ),
            dcc.Dropdown(
                id='food-dropdown',
                options=[{'label': f'At least {i}', 'value': i} for i in sorted(df['food'].dropna().unique())],
                value=None,
                placeholder="Select minimum food score"
            ),
            dcc.Dropdown(
                id='service-dropdown',
                options=[{'label': f'At least {i}', 'value': i} for i in sorted(df['service'].dropna().unique())],
                value=None,
                placeholder="Select minimum service score"
            ),
            dcc.Dropdown(
                id='ambience-dropdown',
                options=[{'label': f'At least {i}', 'value': i} for i in sorted(df['ambience'].dropna().unique())],
                value=None,
                placeholder="Select minimum ambience score"
            ),
            html.Button('Filter Restaurants', id='filter-button', n_clicks=0, style={
                'background-color': '#007BFF', 
                'color': 'white', 
                'padding': '10px', 
                'border': 'none', 
                'border-radius': '5px', 
                'margin-top': '10px'
            }),
        ]),

        # Filtered results and image section on the right
        html.Div(style={'width': '55%', 'display': 'inline-block', 'padding-left': '5%'}, children=[
            html.Div(id='restaurant-info'),
            html.Div(id='summary-output', style={'margin-top': '20px'})
        ]),
    ])
])

# Callback to update restaurant info and display restaurant names as clickable elements
@rest_dash.callback(
    Output('restaurant-info', 'children'),
    Input('filter-button', 'n_clicks'),
    [
        Input('rating-slider', 'value'),
        Input('food-type-dropdown', 'value'),
        Input('price-range-dropdown', 'value'),
        Input('number-reviews-dropdown', 'value'),
    ]
)
def update_restaurant_info(n_clicks, selected_rating, selected_food, selected_price, selected_reviews):
    if n_clicks > 0:  # Only filter when the button is clicked
        filtered_df = df.copy()

        if selected_rating is not None:
            filtered_df = filtered_df[filtered_df['rating'] >= selected_rating]
        
        if selected_food is not None:
            filtered_df = filtered_df[filtered_df['food_type'].str.strip() == selected_food.strip()]
        
        if selected_price is not None:
            filtered_df = filtered_df[filtered_df['coupon'].str.strip() == selected_price.strip()]
        
        if selected_reviews is not None:
            filtered_df = filtered_df[filtered_df['number_of_reviews'] >= selected_reviews]

        if filtered_df.empty:
            return "No matching restaurants found."

        # Display clickable restaurant names with images
        restaurant_cards = [
            html.Div(children=[
                html.H4(row['rest_name'], style={'cursor': 'pointer', 'color': '#007BFF'}, id={'type': 'restaurant-name', 'index': index}, n_clicks=0),
                html.Img(src=row['image_url'], id={'type': 'restaurant-img', 'index': index}, style={'width': '150px', 'height': '150px'}),
            ], style={'margin-bottom': '20px'}) for index, row in filtered_df.iterrows()
        ]
        return restaurant_cards

    return ""

# Callback to summarize comments when a restaurant name is clicked and enlarge the image
@rest_dash.callback(
    [Output('summary-output', 'children'), Output({'type': 'restaurant-img', 'index': dash.dependencies.ALL}, 'style')],
    Input({'type': 'restaurant-name', 'index': dash.dependencies.ALL}, 'n_clicks'),
    Input({'type': 'restaurant-name', 'index': dash.dependencies.ALL}, 'children')
)
def summarize_comments(n_clicks, selected_restaurant):
    enlarged_style = [{'width': '150px', 'height': '150px'}] * len(n_clicks)  # Default image sizes
    
    if any(n_clicks):  # Check if any restaurant has been clicked
        index = n_clicks.index(max(n_clicks))
        selected_restaurant_name = selected_restaurant[index]

        openai.api_key = 'api key'  # Replace with your OpenAI API key
        
        restaurant_comments = df[df['rest_name'] == selected_restaurant_name]['comments'].tolist()
        if not restaurant_comments:
            return "No comments found for this restaurant.", enlarged_style
        
        all_comments = " ".join(restaurant_comments)
        
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant. Summarize the following restaurant reviews."},
                    {"role": "user", "content": all_comments}
                ]
            )
            
            summary = response.choices[0].message["content"]
            enlarged_style[index] = {'width': '300px', 'height': '300px'}  # Enlarge the clicked image
            return f"Summary for {selected_restaurant_name}: {summary}", enlarged_style
        except Exception as e:
            return f"An error occurred while summarizing comments: {str(e)}", enlarged_style
    
    return "Select a restaurant to see comments summary.", enlarged_style

# Run the app on a permanent port with REUSE_PORT
if __name__ == '__main__':
    os.environ['WERKZEUG_RUN_MAIN'] = 'true'
    rest_dash.run_server(debug=True, port=8055, use_reloader=False)